## Data Preparation

### Bounding Boxes

In [ ]:
import pandas as pd
import cv2
import os


def generate_bounding_boxes(csv_path, img_path):
    data = pd.read_csv(csv_path)[:75]
    bboxes = []
    # Read the image
    img = cv2.imread(img_path)

    # Iterate through each row in the CSV and draw the bounding box on the image
    for index, row in data.iterrows():
        x, y, w, h = int(row['x']), int(row['y']), int(row['w']), int(row['h'])
        bboxes.append([[x, y, x + w, y + h], row['text']])

    return bboxes

### Save Cropped Images

In [ ]:
import os
from PIL import Image
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def generate_training_testing():

    root_folder = 'preprocessed_data'

    for fuente_folder in os.listdir(root_folder):
        print('Processing:',fuente_folder)
        for file in os.listdir('preprocessed_data/'+fuente_folder):
            file_path = 'preprocessed_data/'+fuente_folder+'/'+file

            if file.endswith('.jpg'):
                img = Image.open(file_path)
                try:
                    bboxes = generate_bounding_boxes(file_path.replace('.jpg','.csv'), file_path)
                except:
                    continue

                for i, bbox in enumerate(bboxes):
                    try:
                        cropped_img = img.crop(bbox[0])
                        label = bbox[1]
                        file_name = file.replace('.jpg','_{}.jpg'.format(i))
                        cropped_img.save('preprocessed_data/'+'Cropped_Images'+'/'+file_name)
                        with open('data/train_cropped.txt', 'a') as f:
                            f.write(file_name + '//' + str(label) + '\n')
                    except:
                        continue

## Detect Defective Images

In [ ]:
def detect_defective_images():
    root_folder = 'preprocessed_data'
    error_images = []
    for fuente_folder in os.listdir(root_folder):
            print('Processing:',fuente_folder)
            for file in os.listdir('preprocessed_data/Cropped_Images'):
                file_path = 'preprocessed_data/'+'Cropped_Images'+'/'+file

                if file.endswith('.jpg'):
                    try:
                        img = Image.open(file_path)
                    except Exception as e:
                        print('Error:',file_path)
                        print(e)
                        error_images.append(file_path.split('/')[-1])
                        continue
    return error_images

In [ ]:
defective = detect_defective_images()

In [ ]:
df = pd.read_fwf('data/train_cropped.txt', delimiter = '//' ,header=None)



In [ ]:
df[0].apply(lambda x: x.split('//')[0])

In [ ]:
df['1'] = df[0].apply(lambda x: x.split('//')[0])
df[0] = df[0].apply(lambda x: x.split('//')[1])
df[0], df['1'] = df['1'], df[0]

In [ ]:
df.rename(columns={0:'image','1':'text'},inplace=True)
df = df[0:10000]

In [ ]:
mask = df['image'].isin(defective)
df = df[~mask]
df

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class TROCRDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        file_name = self.df['image'][idx]
        text = self.df['text'][idx]

        img = Image.open(self.root_dir + '/' 'Cropped_Images' + '/' + file_name).convert('RGB')
        pixel_values = self.processor(img, return_tensors="pt").pixel_values

        labels = self.processor.tokenizer(text,padding="max_length", max_length=self.max_target_length).input_ids

        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:

from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = TROCRDataset(root_dir='preprocessed_data',
                           df=train_df,
                           processor=processor)
eval_dataset = TROCRDataset(root_dir='preprocessed_data',
                           df=test_df,
                           processor=processor)

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

In [ ]:
encoding = train_dataset[2]
for key, value in encoding.items():
    print(key, value.shape)

In [ ]:
image = Image.open(train_dataset.root_dir + '/' + 'Cropped_Images' + '/' + train_dataset.df['image'][2]).convert('RGB')
image

In [ ]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=4)

## Train Model

In [ ]:
from transformers import VisionEncoderDecoderModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
model.to(device)

In [ ]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

In [ ]:
def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer
     

In [ ]:
from tqdm import tqdm

In [ ]:
from transformers import AdamW
#from tqdm.notebook import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    
   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer 

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

## Unused Functions that might be useful

import os
from PIL import Image
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def generate_training_testing():

    root_folder = 'preprocessed_data'

    for fuente_folder in os.listdir(root_folder):
        print('Processing:',fuente_folder)
        for file in os.listdir('preprocessed_data/'+fuente_folder):
            file_path = 'preprocessed_data/'+fuente_folder+'/'+file
            if file.endswith('.jpg'):
                # write the name of the file in a txt file if it doesn't exit, create it in the data folder
                with open('data/train.txt', 'a') as f:
                    try:
                        vals = pd.read_csv('preprocessed_data/'+fuente_folder+'/'+file.split('.')[0]+'.csv')['text'].values.tolist()
                    except:
                        continue
                    f.write(file + '//' + str(vals) + '\n')

